In [1]:
%pip install transformers torch pandas scikit-learn

%pip install accelerate -U

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your dataset
file_path = '../database/TextEmo/text.csv'  
data = pd.read_csv(file_path)

# Preprocess the text (e.g., lowercasing)
data['text'] = data['text'].str.lower()


In [2]:
data

,id,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4
...,...,...,...
416804,416804,i feel like telling these horny devils to find...,2
416805,416805,i began to realize that when i was feeling agi...,3
416806,416806,i feel very curious be why previous early dawn...,5
416807,416807,i feel that becuase of the tyranical nature of...,3


In [3]:
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    data['text'], data['label'], test_size=0.2, random_state=42
)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42
)

In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text for BERT
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

c:\CodeRepo\DS-Capstone-Spring-2024\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import torch

class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets for training, validation, and testing
train_dataset = EmotionDataset(train_encodings, list(train_labels))
val_dataset = EmotionDataset(val_encodings, list(val_labels))
test_dataset = EmotionDataset(test_encodings, list(test_labels))


In [6]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"There are {torch.cuda.device_count()} GPU(s) available.")
    print("Device name:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("No GPU available, using the CPU instead.")


There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3060 Laptop GPU


In [7]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)
model = model.to(device)  # Move the model to the GPU if available


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

In [15]:
from transformers import TrainingArguments, Trainer
import logging

# Set logging to warning level to reduce output verbosity
logging.getLogger("transformers").setLevel(logging.WARNING)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=1000,
    save_strategy="no",  # No intermediate checkpoints are saved
    evaluation_strategy='epoch',  # Evaluate at the end of each epoch
    load_best_model_at_end=False  # Do not load the best model at the end
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


In [16]:
trainer.train()

  0%|          | 93/125043 [01:37<36:28:15,  1.05s/it]
                                                     
  0%|          | 126/41681 [06:55<3:42:06,  3.12it/s]  

{'loss': 0.6936, 'grad_norm': 9.939903259277344, 'learning_rate': 4.9392923921225805e-05, 'epoch': 0.02}


                                                     
  0%|          | 126/41681 [14:49<3:42:06,  3.12it/s] 

{'loss': 0.3136, 'grad_norm': 0.04120092839002609, 'learning_rate': 4.8178771763677425e-05, 'epoch': 0.05}


                                                     
  0%|          | 126/41681 [23:31<3:42:06,  3.12it/s] 

...

{'loss': 0.1052, 'grad_norm': 0.24577581882476807, 'learning_rate': 2.0409897768388335e-06, 'epoch': 0.96}


                                                       
  0%|          | 126/41681 [4:23:33<3:42:06,  3.12it/s]

{'loss': 0.0909, 'grad_norm': 0.0019743803422898054, 'learning_rate': 8.268376192904496e-07, 'epoch': 0.98}


{'eval_loss': 0.08925820887088776, 'eval_accuracy': 0.9419159809025696, 'eval_runtime': 490.1252, 'eval_samples_per_second': 85.042, 'eval_steps_per_second': 5.317, 'epoch': 1.0}
{'train_runtime': 16531.937, 'train_samples_per_second': 20.17, 'train_steps_per_second': 2.521, 'train_loss': 0.14767046149308233, 'epoch': 1.0}


TrainOutput(global_step=41681, training_loss=0.14767046149308233, metrics={'train_runtime': 16531.937, 'train_samples_per_second': 20.17, 'train_steps_per_second': 2.521, 'train_loss': 0.14767046149308233, 'epoch': 1.0})

In [17]:
trainer.evaluate(test_dataset)

100%|██████████| 2606/2606 [04:44<00:00,  9.15it/s]


{'eval_loss': 0.09024196863174438,
 'eval_accuracy': 0.9414841294594659,
 'eval_runtime': 285.1536,
 'eval_samples_per_second': 146.17,
 'eval_steps_per_second': 9.139,
 'epoch': 1.0}

In [20]:
model.save_pretrained('./text_emotion_detection_model1')
tokenizer.save_pretrained('./text_emotion_detection_model2')


('./text_emotion_detection_model2\\tokenizer_config.json',
 './text_emotion_detection_model2\\special_tokens_map.json',
 './text_emotion_detection_model2\\vocab.txt',
 './text_emotion_detection_model2\\added_tokens.json')

: 